# Assignment 1

## Question A
<b>Links</b>
<nav>
<a href="https://en.wikipedia.org/wiki/List_of_automobile_manufacturers">Wikipedia</a> |
<a href=”https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxuryperformance-sedans”>Forum</a> |
</nav>

### A1. fetch the names of major car companies from the wikipedia page ‘List of automobile manufacturers’.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_automobile_manufacturers"
# Headers to mimic a browser visit
headers = {'User-Agent': 'Mozilla/5.0'}

# Returns a requests.models.Response object
page = requests.get(url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')
#soup

In [3]:
div_section = soup.find_all("div", class_ = 'div-col')
companies = []
for i in div_section:
    comp = i.find_all("a", href = True, title = True)
    for c in comp:
        companies.append(c.get_text())
        
#print("There are total of",len(companies), 'car companies')
companies[:20]

['SNVI',
 'Zanella',
 'Koller',
 'Anasagasti',
 'AutoLatina',
 'Autoar',
 'Crespi',
 'Eniak',
 'Hispano-Argentina',
 'IAVA',
 'IAME',
 'IKA',
 'Sevel Argentina',
 'SIAM Di Tella',
 'ErAZ',
 'ACE EV Group',
 'Borland Racing Developments',
 'Bolwell',
 'Brabham Automotive',
 'Ford Australia']

### A2. Collect ~5000 or more messages posted in Edmunds.com discussion forums on cars.

In [4]:
authors = []
contents = []
dates = []

page = 1
i = 0
while i < 6000:
    url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p'+str(page)
    response = requests.get(url)
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    authors_div = soup.find_all('span', class_ = 'Author')
    for a in authors_div:
        author_id = a.find('a')['data-userid']
        authors.append(author_id)
        
    content_div = soup.find_all('div', class_ = 'Message userContent')
    for c in content_div:
        content = c.get_text()
        contents.append(content)
    
    date_div = soup.find_all('time')
    for d in date_div:
        date = d['datetime']
        dates.append(date)
        i += 1
    page += 1

### A. Save to CSV

In [5]:
df_manufactures = pd.DataFrame({'name': companies})
df_comments = pd.DataFrame({'date': dates, 'userid': authors, 'message': contents})

df_manufactures.to_csv('../data/asm1_manufactures.csv')
df_comments.to_csv('../data/asm1_comments.csv')

In [6]:
print("Manufactures")
display(df_manufactures)
print("Comments")
display(df_comments)

Manufactures


,name
0,SNVI
1,Zanella
2,Koller
3,Anasagasti
4,AutoLatina
...,...
878,Venirauto
879,Thaco
880,Mekong Auto
881,VinFast


Comments


,date,userid,message
0,2002-03-23T19:04:32+00:00,145334,\nEntry level performance luxury sedans are a ...
1,2002-03-25T05:54:02+00:00,409683,\nI personally think that with a few tweaks th...
2,2002-03-25T07:06:29+00:00,224560,\nI am debating a new purchase and these two a...
3,2002-03-25T17:02:27+00:00,75564,"\nGreat handling, RWD, excellent engine and th..."
4,2002-03-25T23:04:37+00:00,274013,\nAnd no manual tranny. That may not matter to...
...,...,...,...
5996,2006-10-03T13:34:02+00:00,88486,"\n When I bought in 1988, fixed mortgage rates..."
5997,2006-10-03T14:16:16+00:00,409608,\nYou won't get an argument from me.I'd never ...
5998,2006-10-03T14:20:50+00:00,409608,"\nAs Rocky point out, this is about SEDANS. La..."
5999,2006-10-03T14:21:33+00:00,410758,"\nRocky, I don't know how you get these ridicu..."


## Question B
Top 5 brands from the forum messages by calculating frequency counts
1. BMW
2. Lexus
3. Acura
4. Infiniti
5. Audi

In [7]:
# return 1 if term t is mentioned in paragraph p
from nltk import wordpunct_tokenize
def t_mentioned_in_p (t, p):
    if ' ' in t:
        return int(t.lower() in p.lower())
    else:
        return int(t.lower() in wordpunct_tokenize(p.lower()))

/Users/admin/opt/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
companies_lower = [x.lower() for x in companies]
df_texts = df_comments
df_texts['message'] = df_texts['message'].apply(lambda x: x.lower())

for c in companies_lower:
    df_texts[c] = df_texts['message'].apply(lambda m: t_mentioned_in_p(c, m))

/var/folders/nv/cw3f8ly11rz_fqddf1glkrz00000gn/T/ipykernel_805/633093245.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_texts[c] = df_texts['message'].apply(lambda m: t_mentioned_in_p(c, m))


In [9]:
comp_counts = [sum(df_texts[c]) for c in companies_lower]
df_comp_pop = pd.DataFrame({'manufacturer': companies, 'count': comp_counts})
df_comp_pop.sort_values(by = 'count', ascending = False).head(10)

,manufacturer,count
238,BMW,1521
425,Lexus,572
414,Acura,483
422,Infiniti,435
236,Audi,363
353,CTS,345
421,Honda,212
430,Nissan,210
567,SEAT,183
333,Standard,150


## Question C
Calculate the frequency of co-mentions of brands  
If multiple brands are mentioned in one message, their mutual co-mention will all be increased.  
Example: If BMW, Lexus, and Infiniti are all mentioned in one comment, the count of (BMW, Lexus), (BMW, Infiniti), and (Lexus, Infiniti) will all be incresed by 1.

The top 3 co-mentioned brands are:
1. Lexus and BMW
3. Infiniti and BMW
3. Acura and BMW

In [10]:
def co_mention(brand1, brand2):
    if brand1 == brand2:
        return 0
    return df_texts[(df_texts[brand1] == 1) & (df_texts[brand2] == 1)].shape[0]

In [11]:
pop_companies = df_comp_pop.sort_values(by = 'count', ascending = False)[:100].manufacturer.to_list()
df_co_mention = {(a,b): co_mention(a.lower(),b.lower()) for a in pop_companies for b in pop_companies}
df_co_mention = pd.DataFrame(df_co_mention.values(), index = df_co_mention.keys())

In [12]:
df_co_mention.sort_values(by=0, ascending=False)[:10]

0
BMW      Lexus     325
Lexus    BMW       325
BMW      Infiniti  239
Infiniti BMW       239
Acura    BMW       226
BMW      Acura     226
         Audi      217
Audi     BMW       217
Lexus    Acura     155
Acura    Lexus     155

## Question D
### D1. find the 5 most frequently mentioned attributes of cars in the discussions.  
The most frequent mentioned attributes are: 'drive', 'perform', 'year', 'seri', 'sport', 'price','engin', 'power', 'test' where seri and engin are the result from stem('series') and stem('engine')

In [13]:
import nltk
from nltk import word_tokenize
from nltk import pos_tag, sent_tokenize, wordpunct_tokenize
from nltk.corpus import wordnet
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
def is_stopword(token):   
    return token.lower() in set(nltk.corpus.stopwords.words('english'))
    
def not_punct(token):
    regular_punct = list(string.punctuation)
    return all([char not in regular_punct for char in token])

cleaned_messages = []
stem = nltk.stem.SnowballStemmer('english')
for m in df_texts['message']:
    x = [stem.stem(c.lower()) for c in wordpunct_tokenize(m) if not_punct(c) and not is_stopword(c)]
    cleaned_message = " ".join(x)
    cleaned_messages.append(cleaned_message)

In [16]:
import numpy.core.multiarray
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Binarizer

cv = CountVectorizer()   
corpus_count = cv.fit_transform(cleaned_messages)    
word_list = cv.get_feature_names()

onehot_scikit = Binarizer()
corpus = onehot_scikit.fit_transform(corpus_count.toarray())

/Users/admin/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
counts_df = pd.DataFrame(corpus, columns = word_list)
counts_df

,00,000,000000,00001,0001,0002353664,000254,000330i,0007,000heat,...,zip,zo6,zone,zoo,zou,zpp,zr,zsp,zumbalak,zx
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
word_counts = [sum(counts_df[w]) for w in word_list]
df_word_freq = pd.DataFrame({'word': word_list, 'count': word_counts})
most_freq_words = df_word_freq.sort_values(by = 'count', ascending = False)['word'].to_list()[:50]
most_freq_words

['car',
 'like',
 'bmw',
 'get',
 'drive',
 'one',
 'would',
 'think',
 'tl',
 'go',
 'look',
 'g35',
 'make',
 'perform',
 'sedan',
 'much',
 'better',
 'even',
 'want',
 'say',
 'know',
 'new',
 'time',
 'year',
 'seri',
 'good',
 'peopl',
 'well',
 'realli',
 'best',
 'price',
 'also',
 'engin',
 'differ',
 'buy',
 'way',
 'still',
 'see',
 'luxuri',
 'handl',
 'thing',
 'take',
 'sport',
 'lexus',
 'come',
 'need',
 'use',
 'power',
 'test',
 'great']

In [19]:
word_list_shorten = ['drive', 'perform', 'year', 'seri', 'sport', 'price','engin', 'power', 'test']
df_word_freq = df_word_freq.sort_values(by = 'count', ascending = False)
df_word_freq[df_word_freq['word'].isin(word_list_shorten)]

,word,count
4986,drive,1433
9625,perform,898
13999,year,775
11243,seri,774
10029,price,696
5288,engin,682
11854,sport,611
9940,power,543
12507,test,538


### D2. Which attributes are most strongly associated with the top 5 brands you found in part A.
The attributes that associate with the top 5 brands the most are 'drive', 'perform', 'seri' and 'year'.

In [20]:
top_5_brands = df_comp_pop.sort_values(by = 'count', ascending = False)['manufacturer'].to_list()[:5]
top_5_brands = [x.lower() for x in top_5_brands]
top_5_brands

['bmw', 'lexus', 'acura', 'infiniti', 'audi']

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
corpus_tfidf = tfidf.fit_transform(cleaned_messages)
word_list_tfidf = tfidf.get_feature_names()
tfidf_df = pd.DataFrame(corpus_tfidf.toarray(), columns = word_list_tfidf)
tfidf_df

/Users/admin/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000000,00001,0001,0002353664,000254,000330i,0007,000heat,...,zip,zo6,zone,zoo,zou,zpp,zr,zsp,zumbalak,zx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
for b in top_5_brands:
    print('Brand:', b)
    b_message_index = df_texts[df_texts[b]==1].index.to_list()
    word_tfidf = [sum(tfidf_df.loc[b_message_index, w]) for w in word_list_shorten]
    word_frequency =[sum(counts_df.loc[b_message_index, w]) for w in word_list_shorten]
    n_messages = df_texts[df_texts[b]==1].shape[0]
    word_frequency_rate = [wf/n_messages for wf in word_frequency]
    df_word_tfidf = pd.DataFrame({'word': word_list_shorten, 'tfidf': word_tfidf, 'frequency': word_frequency, 'rate': word_frequency_rate})
    print(df_word_tfidf.sort_values(by = 'frequency', ascending = False).head(10))
    print()

Brand: bmw
      word      tfidf  frequency      rate
0    drive  42.721105        514  0.337936
3     seri  44.937828        420  0.276134
1  perform  29.556581        339  0.222880
2     year  27.498672        332  0.218277
5    price  25.358145        282  0.185404
6    engin  28.179311        278  0.182774
4    sport  23.333001        251  0.165023
7    power  20.198137        230  0.151216
8     test  18.748547        193  0.126890

Brand: lexus
      word      tfidf  frequency      rate
0    drive  15.133863        190  0.332168
3     seri  12.655217        140  0.244755
1  perform  11.788548        130  0.227273
5    price   9.698376        111  0.194056
6    engin   9.304185        107  0.187063
2     year   8.459155        106  0.185315
4    sport   9.437759        104  0.181818
7    power   5.296684         73  0.127622
8     test   6.562745         67  0.117133

Brand: acura
      word      tfidf  frequency      rate
0    drive  12.010722        163  0.337474
1  perform  10.

## Question E
Which is the most aspirational brand in your data in terms of people actually wanting to buy or own?
<div class="alert alert-block alert-info">
<b>Methedology:</b> Calculate the aspiration Score for each brand by the following formula:  
$$Aspiration Score_{brand} =  \sum \limits _{m}^{Messages_{brand}} (W_m/C_m)$$
Where $W_{m}$ is the number of keywords appeared in the message $m$, 
and $C_m$ is the number of car companies mentioned in the message $m$. So that when multiple brands are mentioned in one post, the score gained by each brand can be diluted.  

The keyword I would like to look at are 'better', 'good', 'best', 'prefer', 'purchas', 'desir', 'dream', 'wish', 'love', 'like', 'get', and 'buy', since they reveal people's willingness of buying the cars.   
    
I chose to count the number of keywords mentioned in one message rather than observing whether any combination of them appeared in one post since as the number of keywords being mentioned increases, the higher the possibility that the message is talking about a dream car.    
</div>

In [23]:
# extract information of word 'better', 'good', 'best', 'prefer', 'purchase', 'desire', 'dream', 'wish', 'love', 'like', 'get', 'buy'
#stem('desire') = desir; stem(purchase) = purchas
arpiration_words = ['better', 'good', 'best', 'prefer', 'purchas', 'desir', 'dream', 'wish', 'love', 'like', 'get', 'buy']
#counts_df[arpiration_words]
aspiration_total_score = counts_df[arpiration_words].apply(lambda x: sum(x), axis = 1)
aspiration_total_score

0       0
1       5
2       2
3       1
4       0
       ..
5996    0
5997    1
5998    0
5999    2
6000    1
Length: 6001, dtype: int64

In [24]:
#co_mention_n_brands: count number of brands mentioned in each message, this will be a penalty added to aspiration score
co_mention_n_brands = df_texts.iloc[:,3:].apply(lambda x: sum(x), axis = 1)
co_mention_n_brands

0        9
1       13
2        3
3        0
4        0
        ..
5996     0
5997     0
5998     1
5999     0
6000     0
Length: 6001, dtype: int64

In [25]:
aspiration_score = []
for c in pop_companies:
    c_index = df_texts[df_texts[c.lower()]==1].index
    msg_scores = aspiration_total_score[c_index]/co_mention_n_brands[c_index]
    score = sum(msg_scores)
    aspiration_score.append(score)
    
df_aspiration = pd.DataFrame({'Brand': pop_companies, 'Aspiration Score': aspiration_score})
df_aspiration.sort_values(by = 'Aspiration Score', ascending = False).head(20)

,Brand,Aspiration Score
0,BMW,1362.492779
1,Lexus,372.104348
2,Acura,309.759796
3,Infiniti,264.437013
4,Audi,248.211142
5,CTS,246.073901
8,SEAT,156.527500
6,Honda,152.092063
7,Nissan,139.945754
9,Standard,132.048352


## Conclusion and Suggestions:
By observing the data scraped from the forum 'Entry Level Luxury Performance Sedans' on Edmunds, we can see that the most popular entry level luxury car brands that people are discussing are <mark>BMW, Lexus, Acura, Infiniti, and Audi </mark>. These brands are usually mentioned together, meaning that buyers typically like to make a comparison of performance between these brands. 
    
According to the most frequently mentioned attributes, other than the main topic of this forum, performance, people also care about the driving experience,  what series is better, their engine power, their year of release, and their last years. Some people might also care about how these cars can support their sports equipment.  
    
The attributes that associate with the top 5 brands the most are <mark>'drive', 'perform', 'seri' and 'year'</mark>. According to the tfidf and frequency scores, we can tell that people like to compare the series of BMW, Lexus, and Infiniti in terms of their performances. For Audi and Acura, people would be more likely to see how long the cars will last. In this forum, people care less about the price compared to performance.  
    
The most aspirational brand for people who want to choose an entry-level luxury car is BMW, followed by Lexus and Acura.
<div class="alert alert-block alert-info">
<b>Suggestions to brand manager:</b> It is better to maintain the reputation of BMW, Lexus, Acure, Infiniti, and Audi while trying to come out with a strategy to promote those with good performance but a lower reputation.</div>
<div class="alert alert-block alert-info">
<b>Suggestions to product manager:</b> Since people care a lot about performance, driving experience, and duration, new products should focus on improving or making innovations on these key features. By looking at the co-mention, the product manager can also investigate each brand's competitors to get some insights.</div>
    <div class="alert alert-block alert-info">
<b>Suggestion to advertising manager:</b> Now we know what brands buyers like and what attributes people care about, therefore, when promoting a new product, we should focus on revealing the competencies in quality and usage scenarios to attract customers.</div>